# Test

In [ ]:
using Yao, QAOA, Random, Combinatorics, Arpack, Distributions, LinearAlgebra, Interpolations
using JLD, HDF5, Printf
using PyPlot
PyPlot.plt.style.use("./paper.mplstyle")

PATH = "/home/ubuntu/Archives/";

In [ ]:
using Revise, SpinFluctuations

In [ ]:
Base.show(io::IO, f::Float64) = @printf(io, "%1.4f", f)

## All data

### Small/Large-gap data

In [ ]:
N = 9
pattern = r"random_SK_instance_N_9_seed_(\d+)\.h5"

# N = 11
# pattern = r"random_SK_instance_N_11_seed_(\d+)\.h5"

# N = 13
# pattern = r"random_SK_instance_N_13_seed_(\d+)\.h5"

# N = 15
# pattern = r"random_SK_instance_N_15_seed_(\d+)\.h5"

# N = 17
# pattern = r"random_SK_instance_N_17_seed_(\d+)\.h5"

# N = 19
# pattern = r"random_SK_instance_N_19_seed_(\d+)\.h5"

# final time for mean-field
# T_final = 10000.
T_final = 32000.

# number of points to get Lyapunov exponent for
npts = 256
npts = 2048

# tolerance for DifferentialEquations.jl when solving mean-field 
tol = 1e-8;

In [ ]:
subdir = "small_gaps"
# subdir = "late_gaps"
folder_name = PATH * @sprintf("data/SK_model/N_%i/%s/", N, subdir)
instance_names = readdir(folder_name);

In [ ]:
# instance_seeds = []
# results_seeds = []

# for instance_name in instance_names
#     seed = match(pattern, instance_name)[1]
#     if occursin("results", instance_name)
#         push!(results_seeds, seed)        
#         continue
#     end
#     push!(instance_seeds, seed)        
# end

# instance_seeds = parse.(Int, instance_seeds)
# results_seeds = parse.(Int, results_seeds);

In [ ]:
couplings_small_gap = Dict()
eigvals_small_gap = Dict()
lyapunov_exponents_small_gap = Dict()

for instance_name in instance_names
    seed = match(pattern, instance_name)[1]
    if occursin("results", instance_name)
        try
            # lyapunov_exponents_small_gap[seed] = h5read(folder_name * instance_name, @sprintf("lyapunov_exponent_T_final_%.0f_tol_1e%.0f_npts_%i", T_final, log10(tol), npts))
            all_flucs = sum(h5read(folder_name * instance_name, @sprintf("fluctuations_T_final_%.0f_tol_1e%.0f_npts_%i", T_final, log10(tol), npts)), dims=1)
            if sum(all_flucs) |> abs < 1e4 
                lyapunov_exponents_small_gap[seed] = all_flucs
            end
        catch e
            printstyled(seed, ": ", e, "\n", color=:red)
        end            
        continue
    end
    try    
        couplings = h5read(folder_name * instance_name, "J")
        couplings_small_gap[seed] = couplings
        eigvals_small_gap[seed] = h5read(folder_name * instance_name, "exact_ARPACK_LM_eigvals")
    catch e
        printstyled(seed, ": ", e, "\n", color=:red)
    end
end


In [ ]:
lyapunov_exponents_small_gap

In [ ]:
subdir = "large_gaps"
# subdir = "extra_data"
folder_name = PATH * @sprintf("data/SK_model/N_%i/%s/", N, subdir)
instance_names = readdir(folder_name)

couplings_large_gap = Dict()
eigvals_large_gap = Dict()
lyapunov_exponents_large_gap = Dict()

for instance_name in instance_names
    seed = match(pattern, instance_name)[1]
    if occursin("results", instance_name)
        # lyapunov_exponents_large_gap[seed] = h5read(folder_name * "results_" * instance_name, @sprintf("lyapunov_exponent_T_final_%.0f_tol_1e%.0f_npts_%i", T_final, log10(tol), npts))    
        lyapunov_exponents_large_gap[seed] = sum(h5read(folder_name * instance_name, @sprintf("fluctuations_T_final_%.0f_tol_1e%.0f_npts_%i", T_final, log10(tol), npts)), dims=1)
        continue
    end
    try
        couplings = h5read(folder_name * instance_name, "J")
        couplings_large_gap[seed] = couplings
        λ = h5read(folder_name * instance_name, "exact_ARPACK_LM_eigvals")
        eigvals_large_gap[seed] = λ
    catch e
        printstyled(seed, ": ", e, "\n", color=:red)
    end                                             
end


In [ ]:
eigvals_small = [v[2, :] .- v[1, :] for (k, v) in eigvals_small_gap]
eigvals_large = [v[2, :] .- v[1, :] for (k, v) in eigvals_large_gap]
avg_eigvals_small = eigvals_small |> mean
avg_eigvals_large = eigvals_large |> mean;

In [ ]:
avg_lyapunov_exponents_small = [v for (k, v) in lyapunov_exponents_small_gap] |> mean
avg_lyapunov_exponents_large = [v for (k, v) in lyapunov_exponents_large_gap] |> mean;

In [ ]:
figure(figsize=(7, 3))
subplot(121)
plot(range(0, 1, 33), avg_eigvals_small, "-o", ms=3)
plot(range(0, 1, 33), avg_eigvals_large, "-s", ms=3)
xlim(0, 1)
ylim(0, 2)
xlabel("\$s\$")
ylabel("Average Exact Gap")

subplot(122)
plot(range(0, 1, npts+1), avg_lyapunov_exponents_small[1, :], "-", ms=3)
plot(range(0, 1, npts+1), avg_lyapunov_exponents_large[1, :], "-", ms=3)
xlim(0, 1)
ylim(0, )
xlabel("\$s\$")
ylabel("Average \$\\lambda\$")
tight_layout()

In [ ]:
length(couplings_small_gap) |> println
length(couplings_large_gap) |> println

In [ ]:
gapsize_small_gap = []
gaplocs_small_gap = []
for (k, v) in eigvals_small_gap
    gap = v[2, :] .- v[1, :]
    minigap = minimum(gap)
    push!(gapsize_small_gap, minigap)
    exact_times = range(0, 1, 33)
    gaploc = exact_times[findfirst(x -> x == minigap, gap)]
    push!(gaplocs_small_gap, gaploc)
end 

In [ ]:
gapsize_large_gap = []
gaplocs_large_gap = []
for (k, v) in eigvals_large_gap
    gap = v[2, :] .- v[1, :]
    minigap = minimum(gap)
    push!(gapsize_large_gap, minigap)
    exact_times = range(0, 1, 33)
    gaploc = exact_times[findfirst(x -> x == minigap, gap)]
    push!(gaplocs_large_gap, gaploc)
end 

In [ ]:
figure(figsize=(7, 3))
ax = subplot(121)
hist2D(gapsize_small_gap, gaplocs_small_gap, bins=32, range=[[0, 1e-2], [0., 1.0]], cmap="gist_earth_r")
colorbar()
xlim(0, 0.01)
ylim(0., 1)
xticks([0., 5e-3, 1e-2])
ax.set_xticklabels([0., "", 1e-2])
xlabel("\$\\Delta\$")
ylabel("\$s_{\\mathrm{min}}\$")

ax = subplot(122)
hist2D(gapsize_large_gap, gaplocs_large_gap, bins=32, range=[[0.5, 1.1], [0., 1.0]], cmap="gist_earth_r")
colorbar()
xlim(0.5, 1.1)
ylim(0.0, 1)
ax.set_yticklabels([])
xlabel("\$\\Delta\$")
# ylabel("\$s_{\\mathrm{min}}\$")
tight_layout()